# LSTM Building Notebook for Next Event Prediction

LSTM (long short-term memory) is a neural network framework built on top of RNNs that allow us to predict the next event based on a given history of events, similar to predicting the next word given a sequence of previous words (one of the most popular applications of LSTMs). This notebook will process our data, train, and evaluate a LSTM model for predicting the next event of a customer's journey. 

### I. Data Processing

In [1]:
import pandas as pd
import numpy as np

# Import initial data
data = pd.read_csv('../smaller_sample.csv')
event_defs = pd.read_csv('../event_definitions.csv')

In [2]:
# Initial data processing here:
from utils import fingerhut_data_cleaner
df = fingerhut_data_cleaner(data, event_defs)

In [3]:
#! TODO: Add journey_id column, customer_quit event, and time since last event variable
#! TODO: Measure baseline success rate for binary classification

In [4]:
df['time_since_last_event'] = df.groupby(['customer_id', 'journey_id'])['event_timestamp'].diff()

In [5]:
df.head()

,customer_id,account_id,ed_id,event_name,event_timestamp,journey_steps_until_end,journey_id,milestone_number,stage,time_since_last_event
0,278713037,1812321640,2,campaign_click,2021-05-31 06:00:00,1,1.0,0.0,Discover,NaT
1,278713037,1812321640,19,application_web_view,2021-05-31 23:11:03,2,1.0,0.0,Apply for Credit,0 days 17:11:03
2,278713037,1812321640,3,application_web_submit,2021-05-31 23:11:51,3,1.0,0.0,Apply for Credit,0 days 00:00:48
3,278713037,1812321640,19,application_web_view,2021-05-31 23:11:51,4,1.0,0.0,Apply for Credit,0 days 00:00:00
4,278713037,1812321640,19,application_web_view,2021-05-31 23:11:54,5,1.0,0.0,Apply for Credit,0 days 00:00:03


In [6]:
from sklearn.model_selection import train_test_split
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

num_classes = df.ed_id.nunique() # 27 (including customer_quit)

In [7]:
df.query("event_name == 'customer_quit'").count()

customer_id                0
account_id                 0
ed_id                      0
event_name                 0
event_timestamp            0
journey_steps_until_end    0
journey_id                 0
milestone_number           0
stage                      0
time_since_last_event      0
dtype: int64

In [8]:
from datetime import timedelta

df['order_shipped'] = df['event_name'] == 'order_shipped'

# Group by customer and journey, then determine if 'order_shipped' occurs in each journey
journey_success = df.groupby(['customer_id', 'journey_id'])['order_shipped'].any().reset_index(name='journey_success')

# Merge this information back to the original DataFrame
df = pd.merge(df, journey_success, how='left', on=['customer_id', 'journey_id'])

# Identify journeys without 'order_shipped' by directly filtering where journey_success is False
journeys_to_quit = df[df['journey_success'] == False].drop_duplicates(subset=['customer_id', 'journey_id'])

# Step 2: For each journey that needs a 'customer_quit' event, add a row 60 days after the last event
new_rows = []
for _, row in journeys_to_quit.iterrows():
    last_event_timestamp = df[(df['customer_id'] == row['customer_id']) & (df['journey_id'] == row['journey_id'])]['event_timestamp'].max()
    new_row = row.copy()
    new_row['event_name'] = 'customer_quit'
    new_row['event_timestamp'] = last_event_timestamp + timedelta(days=60)
    # This line is not needed anymore as we're not using 'order_shipped' in the new rows directly
    # new_row['order_shipped'] = False
    # Reset or adjust any other columns as needed for the new event
    # For example, clear columns that shouldn't carry over to the customer_quit event
    new_rows.append(new_row)

# Append new_rows to the DataFrame
if new_rows:  # Check if there are any rows to add
    df = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)

# Since we're not using 'order_shipped' in the new rows, we can drop the column earlier if you prefer
df.drop(columns=['order_shipped'], inplace=True)

# Sort df by 'customer_id', 'journey_id', and 'event_timestamp' if needed
df.sort_values(by=['customer_id', 'journey_id', 'event_timestamp'], inplace=True)


In [9]:
df.head()

,customer_id,account_id,ed_id,event_name,event_timestamp,journey_steps_until_end,journey_id,milestone_number,stage,time_since_last_event,journey_success
2348005,5414,1995603037,12,application_web_approved,2022-10-09 15:19:51,2,1.0,1.0,Apply for Credit,NaT,False
2348006,5414,1995603037,4,browse_products,2022-10-09 15:29:00,3,1.0,0.0,First Purchase,0 days 00:09:09,False
2348007,5414,1995603037,2,campaign_click,2022-10-09 21:19:51,4,1.0,0.0,Discover,0 days 05:50:51,False
2348008,5414,1995603037,4,browse_products,2022-12-01 11:36:47,5,1.0,0.0,First Purchase,52 days 14:16:56,False
2348009,5414,1995603037,11,add_to_cart,2022-12-01 11:36:49,6,1.0,0.0,First Purchase,0 days 00:00:02,False


In [ ]:
# def create_sequences(df, seq_length=10):
#     X = []
#     y = []
    
#     for _, group in df.groupby(['customer_id', 'journey_id']):
#         # Prepare the sequence of events and steps
#         events = group['ed_id'].tolist()
#         steps = group['journey_steps_until_end'].tolist()
        
#         # Combine events and steps into a single sequence of features
#         for i in range(len(events) - seq_length):
#             sequence = [[events[j], steps[j]] for j in range(i, i + seq_length)]
#             X.append(sequence)
#             y.append(events[i + seq_length])
    
#     return np.array(X), np.array(y)

# seq_length = 10  # Adjust based on your data
# X, y = create_sequences(df, seq_length)

# # Split the dataset
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
df['time_since_last_event'] = df['time_since_last_event'].fillna(pd.Timedelta(seconds=0))

In [13]:
# Assuming 'df' is your DataFrame and you've already filled NaT values with Timedelta(0)
df['time_since_last_event'] = df['time_since_last_event'].dt.total_seconds()

In [14]:
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, TensorDataset

def create_sequences(df, seq_length=10):
    X = []
    y = []
    
    for _, group in df.groupby(['customer_id', 'journey_id']):
        # Extract sequences and the target variable
        events = group['ed_id'].tolist()
        steps_until_end = group['journey_steps_until_end'].tolist()
        time_since_last_event = group['time_since_last_event'].tolist()
        journey_success = group['journey_success'].iloc[-1]  # Assuming binary target is the same for the whole journey
        
        # Combine features into a single sequence for each step
        for i in range(len(events) - seq_length + 1):
            sequence = [[events[j], steps_until_end[j], time_since_last_event[j]] for j in range(i, min(i + seq_length, len(events)))]
            X.append(sequence)
            y.append(journey_success)
    
    # Handling variable sequence lengths: Pad sequences for LSTM input
    X_padded = pad_sequence([torch.tensor(x) for x in X], batch_first=True, padding_value=0)
    y_tensor = torch.tensor(y, dtype=torch.float32)  # Assuming journey_success is a binary variable
    
    return X_padded, y_tensor

# Create sequences with your DataFrame
seq_length = 10  # Adjust based on your data
X, y = create_sequences(df, seq_length)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
from torch.utils.data import Dataset, DataLoader

class EventDataset(Dataset):
    def __init__(self, X, y):
        self.X = X  
        self.y = y
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Create Dataset objects
train_dataset = EventDataset(X_train, y_train)
test_dataset = EventDataset(X_test, y_test)

# Create DataLoader objects
batch_size = 64  # Adjust as per your requirements
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [17]:
X_train.shape

torch.Size([1398868, 10, 3])

In [18]:
# Count the number of successful (1) and unsuccessful (0) journeys in the test set
num_successful = (y_test == 1).sum().item()
num_unsuccessful = (y_test == 0).sum().item()

# Calculate the ratio of successful to unsuccessful journeys
ratio_successful_to_unsuccessful = num_successful / num_unsuccessful

print(f"Number of Successful Journeys: {num_successful}")
print(f"Number of Unsuccessful Journeys: {num_unsuccessful}")
print(f"Ratio of Successful to Unsuccessful Journeys: {ratio_successful_to_unsuccessful}")


Number of Successful Journeys: 133946
Number of Unsuccessful Journeys: 215771
Ratio of Successful to Unsuccessful Journeys: 0.6207785105505373


In [ ]:
# class EventDataset(Dataset):
#     def __init__(self, X, y):
#         self.X = torch.tensor(X, dtype=torch.float32)  # Ensure correct data type
#         self.y = torch.tensor(y, dtype=torch.long)
    
#     def __len__(self):
#         return len(self.X)
    
#     def __getitem__(self, idx):
#         return self.X[idx], self.y[idx]

# # Create Dataset objects
# train_dataset = EventDataset(X_train, y_train)
# test_dataset = EventDataset(X_test, y_test)

# # Create DataLoader objects
# batch_size = 64  # Adjust as per your requirements
# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

### II. Building the Model

In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

class LSTMModelWithSteps(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(LSTMModelWithSteps, self).__init__()
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True)
        # Output layer that outputs the probability of success
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).to(x.device)
        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).to(x.device)
        out, _ = self.lstm(x, (h0.detach(), c0.detach()))
        out = self.fc(out[:, -1, :])  # Use only the last timestep's output
        return out


In [20]:
# Make sure GPU is loaded for training
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using GPU:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("Using CPU")

Using GPU: NVIDIA GeForce RTX 3060


In [21]:
input_dim = 3  # Adjust based on the number of features per timestep
hidden_dim = 100
layer_dim = 1
output_dim = 1  # For binary classification

model = LSTMModelWithSteps(input_dim, hidden_dim, layer_dim, output_dim).to(device)

In [ ]:
# class LSTMModelWithSteps(nn.Module):
#     def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
#         super(LSTMModelWithSteps, self).__init__()
#         self.hidden_dim = hidden_dim
#         self.layer_dim = layer_dim
#         # Input dim is 2 because we now include event and step
#         self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True)
#         self.fc = nn.Linear(hidden_dim, output_dim)
    
#     def forward(self, x):
#         h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).to(x.device)
#         c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).to(x.device)
#         out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
#         out = self.fc(out[:, -1, :])  # We only use the output of the last time step
#         return out


In [ ]:
# # Model instantiation
# input_dim = 2  # Input dimension is the sequence length
# hidden_dim = 100  # Adjust the hidden layer dimension
# layer_dim = 1  # Number of LSTM layers
# output_dim = num_classes  # Number of output classes

# model = LSTMModelWithSteps(input_dim, hidden_dim, layer_dim, output_dim)

### III. Train the Model

In [22]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def validate(model, validation_loader, criterion, device):
    model.eval()
    val_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for sequences, labels in validation_loader:
            sequences, labels = sequences.to(device), labels.to(device).float()
            outputs = model(sequences)
            loss = criterion(outputs.squeeze(), labels)  # Adjust for BCEWithLogitsLoss
            val_loss += loss.item()
            predicted = outputs.squeeze().round()  # Convert probabilities to binary predictions
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    val_loss /= len(validation_loader)
    val_accuracy = 100 * correct / total
    return val_loss, val_accuracy

In [25]:
NUM_EPOCHS = 100

# def validate(model, validation_loader, criterion, device):
#     model.eval()  # Set the model to evaluation mode
#     val_loss = 0
#     correct = 0
#     total = 0
#     with torch.no_grad():  # No need to track gradients during validation
#         for sequences, labels in validation_loader:
#             sequences, labels = sequences.to(device), labels.to(device)
#             outputs = model(sequences)
#             loss = criterion(outputs, labels)
#             val_loss += loss.item()
#             _, predicted = torch.max(outputs.data, 1)
#             total += labels.size(0)
#             correct += (predicted == labels).sum().item()
#     val_loss /= len(validation_loader)
#     val_accuracy = 100 * correct / total
#     return val_loss, val_accuracy

for epoch in range(NUM_EPOCHS):
    model.train()
    loop = tqdm(enumerate(train_loader), total=len(train_loader), leave=True)
    for i, (sequences, labels) in loop:
        sequences, labels = sequences.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(sequences).squeeze()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        loop.set_description(f"Epoch [{epoch+1}/{NUM_EPOCHS}]")
        loop.set_postfix(loss=loss.item())

    # Validation phase (once per epoch)
    val_loss, val_accuracy = validate(model, test_loader, criterion, device)
    print(f'Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%')

Epoch [1/100]: 100%|██████████| 21858/21858 [01:32<00:00, 235.47it/s, loss=0.616]


Validation Loss: 0.6192, Validation Accuracy: 12.39%


Epoch [2/100]: 100%|██████████| 21858/21858 [01:29<00:00, 244.43it/s, loss=0.721]


Validation Loss: 0.6127, Validation Accuracy: 19.05%


Epoch [3/100]: 100%|██████████| 21858/21858 [01:25<00:00, 254.87it/s, loss=0.73] 


...

Validation Loss: 0.6058, Validation Accuracy: 33.60%


Epoch [46/100]: 100%|██████████| 21858/21858 [01:22<00:00, 265.92it/s, loss=0.659]


Validation Loss: 0.6064, Validation Accuracy: 16.59%


Epoch [47/100]:  81%|████████▏ | 17805/21858 [28:11<06:25, 10.53it/s, loss=0.607]   


KeyboardInterrupt: 

In [ ]:
# Save only the model parameters (recommended)
torch.save(model.state_dict(), 'lstm_predictor.pth')

### IV. Inference

In [ ]:
# Rerun Part II (Building the Model) to define the model class if necessary
# model = LSTMModelWithSteps(input_dim, hidden_dim, layer_dim, output_dim).to(device)

In [ ]:
# Load weights into model (only necessary if you started a new session)
model.load_state_dict(torch.load('lstm_predictor.pth'))

In [ ]:
def predict(model, test_loader, device):
    model.eval()  # Ensure the model is in evaluation mode
    predictions = []
    with torch.no_grad():  # No need to track gradients
        for sequences, _ in test_loader:  # Assuming you don't need labels just for predictions
            sequences = sequences.to(device)
            outputs = model(sequences)
            _, predicted = torch.max(outputs.data, 1)  # Get the index of the max log-probability as the prediction
            predictions.extend(predicted.cpu().numpy())  # Move predictions to CPU and convert to numpy, then extend the list
    return predictions

In [ ]:
# Make predictions
predictions = predict(model, test_loader, device)

In [ ]:
# Convert predictions list to a numpy array
predictions_array = np.array(predictions)

# Use inverse_transform to convert numerical labels back to original event names
event_names = le.inverse_transform(predictions_array)